In [2]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sahilsiddharth/DataScienceProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sahilsiddharth"
os.environ["MLFLOW_TRACKING_PASSWORD"]="65a3e985f82ad56f551a6c1e7b4eeaa226c76d7c"
os.environ["REQUESTS_CA_BUNDLE"]=""
import mlflow
#mlflow.set_registry_uri("https://dagshub.com/sahilsiddharth/DataScienceProject.mlflow")

In [3]:
import os
%pwd
os.chdir("../")
%pwd

'c:\\Users\\t0263519\\Desktop\\Study\\PYTHON\\MLFLOW-Projects\\DataScienceProject-1'

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    all_params: str



In [5]:
from src.DataScience.constants import *
from src.DataScience.utils.common import *

import os
import pandas as pd


In [6]:
class ConfigurationManager():
    def __init__(self, 
                 config_file_path=CONFIG_FILE_PATH, 
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
       
        
        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="http://127.0.0.1:5000"
    
            

        )
        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlflow.models.signature import infer_signature



In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        #mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        input_example = test_x.iloc[:5]
        signature = infer_signature(input_example)

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            print(scores)
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model",signature=signature,input_example=input_example)


In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-02-11 12:04:20,868: INFO: common: yaml file config\config.yaml loaded successfully:]
[2025-02-11 12:04:20,869: INFO: common: yaml file params.yaml loaded successfully:]
[2025-02-11 12:04:20,872: INFO: common: yaml file schema.yaml loaded successfully:]
[2025-02-11 12:04:20,873: INFO: common:  directory artifacts created:]
[2025-02-11 12:04:20,874: INFO: common:  directory artifacts/model_evaluation created:]
[2025-02-11 12:04:23,234: WARNING: connectionpool: Retrying (Retry(total=6, connect=6, read=7, redirect=7, status=7)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024DEE4EB130>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')': /api/2.0/mlflow/runs/create:]
[2025-02-11 12:04:29,629: WARNING: connectionpool: Retrying (Retry(total=5, connect=5, read=7, redirect=7, status=7)) after connection broken by 'NewConnectionError('<urllib3.connection.

2025/02/11 12:05:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2025/02/11 12:05:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1


🏃 View run agreeable-bat-717 at: http://127.0.0.1:5000/#/experiments/0/runs/ba4c557207a64b75839671bcc951a691
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'ElasticnetModel'.


In [10]:
import certifi
print(certifi.where())


c:\Users\t0263519\Desktop\Study\PYTHON\MLFLOW-Projects\DataScienceProject\venv\lib\site-packages\certifi\cacert.pem


In [11]:
REQUESTS_CA_BUNDLE=$(python -c "import certifi; print(certifi.where())")


SyntaxError: invalid syntax (4148547993.py, line 1)